---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
# G = P1_Graphs[3]
# degrees = G.degree()
# degree_values = sorted(set(degrees.values()))
# histogram =[list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
# nx.average_shortest_path_length(G)
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.plot(degree_values,histogram, 'o')
# plt.xlabel('Degree')
# plt.ylabel('Fraction of Nodes')

# plt.tight_layout()
# plt.show()

In [4]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']# Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [15]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [50]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(n_estimators=20).fit(X_train,y_train.reshape(-1,1))
# clf.score(X_test,y_test.reshape(-1,1))
# from sklearn.naive_bayes import GaussianNB
# clf2 = GaussianNB().fit(X_train,y_train.reshape(-1,1))
# clf2.score(X_test,y_test.reshape(-1,1))
# from sklearn.tree import DecisionTreeClassifier

# clf3 = DecisionTreeClassifier(random_state=0).fit(X_train,y_train.reshape(-1,1))
# clf3.score(X_test,y_test.reshape(-1,1))
# from sklearn.neighbors import KNeighborsClassifier
# clf5 = KNeighborsClassifier(n_neighbors=10).fit(X_train,y_train.reshape(-1,1))
# clf4.score(X_test,y_test.reshape(-1,1))
# clf5.score(X_test,y_test.reshape(-1,1))

In [6]:

# df['closeness'] = pd.Series(list(nx.closeness_centrality(G).values()))
# df['degree'] = pd.Series(list(G.degree().values()))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
# clf_1 = LogisticRegression().fit(X_train.reshape(-1,1),y_train.reshape(-1,1))
# clf_2 = KNeighborsClassifier(n_neighbors=10).fit(X_train.reshape(-1,1),y_train.reshape(-1,1))
# clf_3 = GaussianNB().fit(X_train.reshape(-1,1),y_train.reshape(-1,1))
# clf_1.score(X_test.reshape(-1,1),y_test.reshape(-1,1))
# clf_2.score(X_test.reshape(-1,1),y_test.reshape(-1,1))
# clf_3.score(X_test.reshape(-1,1),y_test.reshape(-1,1))

In [74]:
def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    df['salary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df['degree'] = list(G.degree().values())
    df['closeness'] = list(nx.closeness_centrality(G).values())
    df['degree centrality'] = list(nx.degree_centrality(G).values())
    df['page rank'] = list(nx.pagerank(G).values())
    df['hubs'] = list(nx.hits(G)[0].values())
    df['auth'] = list(nx.hits(G)[1].values())
    df = df.replace(np.nan, 99)
    test_df = df.where(df['salary'] == 99).dropna()
    test_df = test_df.drop('salary',axis=1)
    train_df = df.where(df['salary']!=99).dropna()
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(train_df[['department', 'degree', 'closeness', 'degree centrality',
           'page rank', 'hubs', 'auth']],train_df['salary'],stratify=train_df['salary'],test_size=0.20 ,random_state=0)
    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression().fit(X_train,y_train.reshape(-1,1))
    ans2 = pd.Series(clf.predict_proba(test_df)[:,1])
    ans2.index = test_df.index
    
    # Your Code Here
    
    return ans2# Your Answer Here

In [75]:
#salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1       0.176274
2       0.588996
5       0.964197
8       0.127189
14      0.391285
18      0.198028
27      0.271782
30      0.309767
31      0.186144
34      0.101334
37      0.123778
40      0.251080
45      0.187280
54      0.217113
55      0.169419
60      0.208332
62      0.994251
65      0.492075
77      0.073590
79      0.139017
97      0.087787
101     0.100230
103     0.188194
108     0.143409
113     0.475363
122     0.076866
141     0.452830
142     0.909547
144     0.084399
145     0.412081
          ...   
913     0.074906
914     0.079059
915     0.036228
918     0.084496
923     0.041994
926     0.064781
931     0.069386
934     0.054550
939     0.035884
944     0.042421
945     0.040195
947     0.046617
950     0.210184
951     0.053288
953     0.060370
959     0.032521
962     0.036453
963     0.158552
968     0.042163
969     0.039158
974     0.040409
984     0.052744
987     0.060709
989     0.038310
991     0.052791
992     0.034736
994     0.040870
996     0.0375

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [9]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
#future_connections.corr()


In [11]:
#model1.score(Xt_test,yt_test)

In [12]:
# yt_pred = model1.predict(Xt_test)

In [13]:
# from sklearn.metrics import confusion_matrix
# confusion_matrix(yt_test,yt_pred)

In [14]:
def new_connections_predictions():
    lst = []
    cmnneighbors = []
    pa_lst = []
    cof_lst = []
    resalloc_lst = []
    AdamicIndex_lst = []
    for i in future_connections.index:
        try:
            lst.append(len(nx.shortest_path(G,i[0],i[1])))
        except:
            lst.append(0)
            continue


    for i in future_connections.index:
        try:
            cmnneighbors.append(len(list(nx.common_neighbors(G,i[0],i[1]))))
        except:
            cmnneighbors.append(0)
            continue


    for u,v,p in nx.preferential_attachment(G, list(future_connections.index)):
        pa_lst.append(p)

    for u,v,p in nx.jaccard_coefficient(G, list(future_connections.index)):
        cof_lst.append(p)



    for u,v,p in nx.resource_allocation_index(G, list(future_connections.index)):
        resalloc_lst.append(p) 


    for u,v,p in nx.adamic_adar_index(G, list(future_connections.index)):
        AdamicIndex_lst.append(p)

    future_connections['length'] = lst
    future_connections['common neighbors'] = cmnneighbors
    future_connections['preferential attachment'] = pa_lst
    future_connections['Jaccard Coefficient'] = cof_lst
    future_connections['Resource Allocation Index'] = resalloc_lst
    future_connections['Adamic-Adar Index'] = AdamicIndex_lst
    future_connections['Future Connection'] = future_connections['Future Connection'].replace(np.nan, 99)
    traindata = future_connections.where(future_connections['Future Connection']!=99).dropna()
    testdata = future_connections.where(future_connections['Future Connection']==99).dropna()
    # Your Code Here
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    Xt_train, Xt_test, yt_train, yt_test = train_test_split(traindata[['length', 'common neighbors',
       'Jaccard Coefficient', 'preferential attachment',
       'Resource Allocation Index', 'Adamic-Adar Index']], traindata['Future Connection'], stratify=traindata['Future Connection'], test_size=0.40,random_state=0)
    model1 = RandomForestClassifier(n_estimators=20).fit(Xt_train,yt_train)
    ans3 = pd.Series(model1.predict_proba(testdata[['length', 'common neighbors',
           'Jaccard Coefficient', 'preferential attachment',
           'Resource Allocation Index', 'Adamic-Adar Index']])[:,1])
    ans3.index = testdata.index
    
    return ans3# Your Answer Here